# Function Calling example

## 概述
本文件主要展示了如何使用LLM 调用外部工具的示例。函数调用（Function Calling）允许大语言模型与外部API和工具进行交互，扩展模型的能力边界。

## 核心概念
- **工具定义**：通过JSON Schema定义LLM可以调用的外部工具
- **工具调用**：LLM识别用户意图并决定调用哪个工具
- **工具执行**：实际执行外部工具并获取结果
- **结果整合**：LLM基于工具返回结果生成最终回答

## 应用场景
- 天气查询、股票信息获取等实时数据查询
- 数据库操作、API调用等系统集成
- 计算器、单位转换等工具类应用
- 文件操作、系统命令执行等本地操作

参考文档 [Deepseek Function Calling](https://api-docs.deepseek.com/zh-cn/guides/function_calling) 

## 导入依赖包

本部分导入实现函数调用功能所需的Python包：
- `openai`: 用于调用DeepSeek等兼容OpenAI API的LLM服务
- `dotenv`: 用于从环境变量文件加载API密钥
- `os`: 用于访问系统环境变量
- `tools.message`: 自定义的消息处理工具模块

In [18]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import tools.message as message_tool

## 初始化llm 客户端

本部分初始化DeepSeek LLM客户端：
- 使用`load_dotenv()`从.env文件加载环境变量
- 获取DeepSeek API密钥
- 配置OpenAI客户端连接到DeepSeek API端点

In [6]:
load_dotenv()
api_key = os.getenv("DEEPSEEK_API_KEY")
client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com",
)

## 定义调用llm 的函数

本部分定义与LLM交互的核心函数，包括外部工具定义和消息发送函数。这是实现函数调用的关键部分。

### 定义外部工具

定义LLM可以调用的外部工具。这里定义了一个天气查询工具：
- 工具名称：`get_weather`
- 功能描述：查询指定地点的天气信息
- 必需参数：`location`（城市和州，如"San Francisco, CA"）

In [7]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather of a location, the user should supply a location first.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"]
            },
        }
    },
]

### 定义消息发送函数

定义发送消息到LLM的函数：
- 使用DeepSeek Chat模型
- 支持工具调用功能
- 返回模型生成的响应消息

In [8]:
def send_messages(messages):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        tools=tools
    )
    return response.choices[0].message

## 调用llm 对话

本部分演示完整的函数调用流程：
- 用户提问
- LLM识别需要调用工具
- 模拟工具执行
- LLM基于工具结果生成最终回答

### 发送问题，杭州的天气怎么样

In [9]:
messages = [{"role": "user", "content": "How's the weather in Hangzhou, Zhejiang?"}]
message = send_messages(messages)
print(f"User>\t {messages[0]['content']}")
messages.append(message)

User>	 How's the weather in Hangzhou, Zhejiang?


In [13]:
print(f"Model>\t {message}")
print(f"Model Tool Call>\t {message.tool_calls}")

Model>	 ChatCompletionMessage(content="I'll check the weather in Hangzhou, Zhejiang for you.", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_00_EeAMg2oCQ3grbgdrF6HezgEv', function=Function(arguments='{"location": "Hangzhou, Zhejiang"}', name='get_weather'), type='function', index=0)])
Model Tool Call>	 [ChatCompletionMessageFunctionToolCall(id='call_00_EeAMg2oCQ3grbgdrF6HezgEv', function=Function(arguments='{"location": "Hangzhou, Zhejiang"}', name='get_weather'), type='function', index=0)]


### mock tool 调用

模拟工具执行过程：
- 获取LLM返回的工具调用信息
- 模拟执行天气查询工具
- 将工具执行结果添加到对话历史中

In [ ]:
tool = message.tool_calls[0]
messages.append({"role": "tool", "tool_call_id": tool.id, "content": "24℃"})


In [15]:
print(f"{messages}")

[{'role': 'user', 'content': "How's the weather in Hangzhou, Zhejiang?"}, ChatCompletionMessage(content="I'll check the weather in Hangzhou, Zhejiang for you.", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_00_EeAMg2oCQ3grbgdrF6HezgEv', function=Function(arguments='{"location": "Hangzhou, Zhejiang"}', name='get_weather'), type='function', index=0)]), {'role': 'tool', 'tool_call_id': 'call_00_EeAMg2oCQ3grbgdrF6HezgEv', 'content': '24℃'}]


### 将 tool 调用的结果发送给llm

将工具执行结果发送回LLM：
- LLM基于工具返回的数据生成最终回答
- 完成完整的函数调用流程
- 展示LLM如何整合外部工具信息

In [20]:
message = send_messages(messages)
print(f"Model>\t {message.content}")
messages.append(message)

Model>	 The current weather in Hangzhou, Zhejiang is 24°C. It's a pleasant temperature - not too hot and not too cold. Would you like any additional weather information about Hangzhou?


In [21]:
message_tool.print_messages(messages)

User: How's the weather in Hangzhou, Zhejiang?

Assistant: I'll check the weather in Hangzhou, Zhejiang for you.

Tool: 24℃

Assistant: The current weather in Hangzhou, Zhejiang is 24°C. It's a pleasant temperature - not too hot and not too cold. Would you like any additional weather information about Hangzhou?

